# Networks and empathy - what matters for my well-being?

* tl;dr - in none of these analyses does your, or anyone else's empathy matter for your well being. For any measure of well being (life satisfaction, stress, loneliness, or composite) all that really matters at t2 is your value of that variable at t1.
* The only significant finding that I trust is that a high IN-degree in your UNION network is associated with being less lonely (p = .001). All other p-values are nonsignificant or marginally significant, and thereby ignorable considering the massive multiple testing here.

**Author: Everett Wetchler (`everett.wetchler@gmail.com`)**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Configuration" data-toc-modified-id="Configuration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configuration</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Cleaning---drop-rows-we-don't-want" data-toc-modified-id="Cleaning---drop-rows-we-don't-want-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cleaning - drop rows we don't want</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model</a></span></li></ul></div>

## Configuration

In [1]:
DVS = c('wellbeing_composite', 'life_satisfaction', 'stress', 'loneliness')
DVS

[1] "wellbeing_composite" "life_satisfaction"   "stress"             
[4] "loneliness"

## Libraries

In [2]:
library(car)
library(tidyverse)
library(mice)
library(nlme)
library(lme4)
library(lmerTest)

Loading required package: carData

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::recode() masks car::recode()
✖ purrr::some()   masks car::some()

Loading required package: lattice

Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 


Attaching package: ‘mice’


The following object is masked from ‘package:tidyr’:

    complete


The following objects are masked from ‘package:base’:

    cbind, rbind



Attaching package: ‘nlme’


The following object is mask

In [3]:
df = read.csv('../data/2019–2020/postprocessed/df_Rcleaned_train.csv', na.strings=c(""," ", "NA"))
df = df[, -1]  # Drop first column (a meaningless row number)
dim(df)
head(df)

[1] 560  60

,PID,gender,race,dorm,life_satisfaction_t1,empathy,loneliness_t1,stress_t1,BFI_E,BFI_A,⋯,empathy_NegAff,degree_in_Gossip,degree_out_Gossip,empathy_Gossip,degree_in_Liked,degree_out_Liked,empathy_Liked,degree_in_StudyWith,degree_out_StudyWith,empathy_StudyWith
,<int>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>
1,1001,male,south_asian,Rinconada,4.833333,6.375,1.333333,2.5,6.5,6.0,⋯,NA,1,0,NA,1,0,NA,1,0,NA
2,1047,female,other_or_mixed,Twain,5.333333,6.500,2.000000,2.5,5.5,6.5,⋯,6.5,4,3,6.541667,1,2,6.000,2,1,6.6250
3,1078,female,east_asian,Loro,5.500000,6.250,1.333333,2.0,4.5,6.5,⋯,5.0,0,0,NA,0,0,NA,0,0,NA
4,1097,male,east_asian,Otero,6.000000,5.000,2.000000,2.0,3.0,4.0,⋯,NA,0,1,5.625000,0,1,6.000,3,1,6.6250
5,1105,female,white,Larkin,6.666667,5.750,2.000000,2.0,4.5,5.0,⋯,NA,2,3,5.750000,1,1,5.125,4,2,6.1875
6,1110,female,white,Larkin,5.166667,6.750,2.000000,2.0,6.5,5.0,⋯,NA,3,4,5.406250,4,1,6.000,3,4,5.4375


In [4]:
cor(df[c('life_satisfaction_t1', 'stress_t1', 'loneliness_t1', 'wellbeing_composite_t1',
         'life_satisfaction_t2', 'stress_t2', 'loneliness_t2', 'wellbeing_composite_t2')],
         use="complete.obs")

,life_satisfaction_t1,stress_t1,loneliness_t1,wellbeing_composite_t1,life_satisfaction_t2,stress_t2,loneliness_t2,wellbeing_composite_t2
life_satisfaction_t1,1.0000000,-0.3589387,-0.4399577,0.7970520,0.7297627,-0.2386400,-0.3011955,0.5603581
stress_t1,-0.3589387,1.0000000,0.2307441,-0.7068405,-0.3362168,0.4599007,0.2220392,-0.4511718
loneliness_t1,-0.4399577,0.2307441,1.0000000,-0.7453631,-0.3924081,0.2041712,0.5091806,-0.4915647
wellbeing_composite_t1,0.7970520,-0.7068405,-0.7453631,1.0000000,0.6467910,-0.4014562,-0.4600154,0.6680755
life_satisfaction_t2,0.7297627,-0.3362168,-0.3924081,0.6467910,1.0000000,-0.3161997,-0.4345107,0.7728812
stress_t2,-0.2386400,0.4599007,0.2041712,-0.4014562,-0.3161997,1.0000000,0.2875269,-0.7118855
loneliness_t2,-0.3011955,0.2220392,0.5091806,-0.4600154,-0.4345107,0.2875269,1.0000000,-0.7682709
wellbeing_composite_t2,0.5603581,-0.4511718,-0.4915647,0.6680755,0.7728812,-0.7118855,-0.7682709,1.0000000


## Cleaning - drop rows we don't want

In [5]:
networks = c()
for (col in names(df)) {
    if (startsWith(col, 'degree_out')) {
        networks = c(networks, strsplit(col, "_")[[1]][[3]])
    }
}
networks

[1] "UNION"        "INTIMATE"     "ACQUAINTANCE" "CloseFrds"    "NegEmoSupp"  
 [6] "PosEmoSupp"   "Responsive"   "EmpSupp"      "PosAff"       "NegAff"      
[11] "Gossip"       "Liked"        "StudyWith"

In [6]:
msg = colSums(is.na(df))
msg = msg[msg > 0]
msg

gender                     race                    BFI_C 
                       3                        3                        1 
            intl_student            family_income     life_satisfaction_t2 
                       3                       20                        1 
           loneliness_t2                stress_t2 parent_education_highest 
                       2                        2                       39 
  wellbeing_composite_t2            empathy_UNION         empathy_INTIMATE 
                       3                       51                       84 
    empathy_ACQUAINTANCE        empathy_CloseFrds       empathy_NegEmoSupp 
                     284                       98                      181 
      empathy_PosEmoSupp       empathy_Responsive          empathy_EmpSupp 
                     174                      216                      249 
          empathy_PosAff           empathy_NegAff           empathy_Gossip 
                     162                      430                      202 
           empathy_Liked        empathy_StudyWith 
                     246                      228

## Model

In [7]:
for (DV in DVS) {
    print("******************************************")
    print(paste("*******", DV, sep=' '))
    print("******************************************")
    DV_t1 = paste(DV, 't1', sep='_')
    DV_t2 = paste(DV, 't2', sep='_')
    print("******************************************")
    print("** BASE MODEL (predict t2 from t1) **")
    print("******************************************")
    print(summary(lm(as.formula(paste(DV_t2, "~", DV_t1, "+ empathy", sep=' ')), data=df)))
    for (net in networks) {
        print("******************************************")
        print(paste("**", DV, "by", net, sep=" "))
        print("******************************************")
        equation = paste(
            DV_t2, ' ~ ', DV_t1, ' + degree_in_',
            net, ' + degree_out_', net, ' + empathy_', net, sep='')
        print(equation)
        print(summary(lm(as.formula(equation), data=df)))
        print("")
    }
}

[1] "******************************************"
[1] "******* wellbeing_composite"
[1] "******************************************"
[1] "******************************************"
[1] "** BASE MODEL (predict t2 from t1) **"
[1] "******************************************"

Call:
lm(formula = as.formula(paste(DV_t2, "~", DV_t1, "+ empathy", 
    sep = " ")), data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.2817 -0.4571  0.1080  0.4734  2.1327 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)             0.21497    0.26859   0.800    0.424    
wellbeing_composite_t1  0.66914    0.03162  21.162   <2e-16 ***
empathy                -0.04002    0.04665  -0.858    0.391    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7402 on 554 degrees of freedom
  (3 observations deleted due to missingness)
Multiple R-squared:  0.4471,	Adjusted R-squared:  0.4451 
F-statistic:   224 on 2 and 554 D

In [9]:
## One-off additional analysis: intimate + acquaintence networks together

In [12]:
for (DV in DVS) {
    print("******************************************")
    print(paste("*******", DV, sep=' '))
    print("******************************************")
    DV_t1 = paste(DV, 't1', sep='_')
    DV_t2 = paste(DV, 't2', sep='_')
    equation = paste(
        DV_t2, ' ~ ', DV_t1,
        ' + degree_in_NegEmoSupp + degree_out_NegEmoSupp + empathy_NegEmoSupp',
        ' + degree_in_ACQUAINTANCE + degree_out_ACQUAINTANCE + empathy_ACQUAINTANCE',
        sep='')
    print(equation)
    print(summary(lm(as.formula(equation), data=df)))
    print("")
}

[1] "******************************************"
[1] "******* wellbeing_composite"
[1] "******************************************"
[1] "wellbeing_composite_t2 ~ wellbeing_composite_t1 + degree_in_NegEmoSupp + degree_out_NegEmoSupp + empathy_NegEmoSupp + degree_in_ACQUAINTANCE + degree_out_ACQUAINTANCE + empathy_ACQUAINTANCE"

Call:
lm(formula = as.formula(equation), data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.4133 -0.4891  0.0774  0.4224  1.7023 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)             -0.2703655  0.8334808  -0.324    0.746    
wellbeing_composite_t1   0.5764059  0.0581082   9.920   <2e-16 ***
degree_in_NegEmoSupp     0.0547064  0.0350744   1.560    0.121    
degree_out_NegEmoSupp   -0.0485290  0.0562383  -0.863    0.389    
empathy_NegEmoSupp       0.0632288  0.1046587   0.604    0.547    
degree_in_ACQUAINTANCE   0.0007858  0.0433166   0.018    0.986    
degree_out_ACQUAINTANCE  0.0568658  0.080